In [1]:
from flair.models import TextClassifier
from flair.data import Sentence, Token
from flair.models import SequenceTagger


############### Gene & Disease NER ######################
# load the model you trained
gene_model = SequenceTagger.load('../flair_models/ner/multi_bio_ner_model_gene/best-model.pt')
disease_model = SequenceTagger.load('../flair_models/ner/multi_bio_ner_model_disease/best-model.pt')

############### Gene & Disease Association ######################
# load the model you trained


association_classifier = TextClassifier.load('../flair_models/text_classification/association_model'+'/best-model.pt')

############### Gene & Disease Sentiment ######################
# load the model you trained

sentiment_classifier = TextClassifier.load('../flair_models/text_classification/sentiment_model'+'/best-model.pt')


2019-09-23 13:33:07,114 loading file ../flair_models/ner/multi_bio_ner_model_gene/best-model.pt
2019-09-23 13:33:26,382 loading file ../flair_models/ner/multi_bio_ner_model_disease/best-model.pt
2019-09-23 13:33:33,789 loading file ../flair_models/text_classification/association_model/best-model.pt
2019-09-23 13:34:16,561 loading file ../flair_models/text_classification/sentiment_model/best-model.pt


In [2]:
import os
import pandas as pd
# import glob
import json
import re

colNames = ('pmcid', 'section', 'target', 'disease', 'text', 'ml_target', 'ml_disease', 'association', 'sentiment' )

file_path = '/nfs/gns/literature/Santosh_Tirunagari/OTAR_DUMPS/Toy/Mixed_Json'
# all_files = glob.glob(file_path+'*split*')

result_folder = '/nfs/gns/literature/Santosh_Tirunagari/OTAR_Results/Results/'



In [ ]:
import re

json_string = '144:{"type":"literature","sourceID":"europepmc","target":{"activity":"http://identifiers.org/cttv.activity/up_or_down","id":"http://identifiers.org/uniprot/O60682","target_type":"http://identifiers.org/cttv.target/protein_evidence"},"disease":{"id":"http://www.ebi.ac.uk/efo/EFO_0000616","name":"tumor"},"evidence":{"unique_experiment_reference":"http://europepmc.org/abstract/MED/16168115","resource_score":{"type":"summed_total","value":40.6,"method":{"description":"Custom text-mining method for target-disease association"}},"literature_ref":{"lit_id":"http://europepmc.org/abstract/MED/16168115","mined_sentences":[{"section":"abstract","t_start":37,"t_end":39,"d_start":127,"d_end":131,"text":"We have demonstrated previously that MSC induces apoptosis after a cell growth arrest in S phase in a mouse mammary epithelial tumor cell model (TM6 cells) in vitro."},{"section":"abstract","t_start":160,"t_end":162,"d_start":118,"d_end":122,"text":"The present study was designed to examine the involvement of the phosphatidylinositol 3-kinase (PI3-K) pathway in TM6 tumor model in vitro after treatment with MSC."},{"section":"abstract","t_start":27,"t_end":29,"d_start":83,"d_end":87,"text":"These studies suggest that MSC blocks multiple signaling pathways in mouse mammary tumor cells."},{"section":"abstract","t_start":0,"t_end":2,"d_start":116,"d_end":120,"text":"MSC inhibits cell growth by inhibiting the activity of PI3-K and its downstream effector molecules in mouse mammary tumor cells in vitro."},{"section":"introduction_and_background","t_start":28,"t_end":30,"d_start":15,"d_end":19,"text":"In the mammary tumor model, MSC is more efficacious than the most extensively studied selenoamino acids in animal models [15,18]."},{"section":"introduction_and_background","t_start":13,"t_end":15,"d_start":63,"d_end":67,"text":"Furthermore, MSC inhibits cell growth in several mouse mammary tumor cell lines [19,20] and human breast cancer cell lines [21]."},{"section":"introduction_and_background","t_start":57,"t_end":59,"d_start":233,"d_end":237,"text":"For the present investigation we examined the effects of MSC on the components of the PI3-KâAkt and RafâMEKâERK pathways to improve our understanding of the mechanisms of growth inhibition in the synchronized TM6 mouse mammary tumor cell line."},{"section":"results","t_start":15,"t_end":17,"d_start":130,"d_end":134,"text":"Treatment with MSC inhibited DNA synthesis in both asynchronous (Fig. 2a) and synchronized (Fig. 2b) TM6 mouse mammary epithelial tumor cells, as measured by [3H]thymidine incorporation."},{"section":"results","t_start":0,"t_end":2,"d_start":44,"d_end":48,"text":"MSC induces apoptosis in mammary epithelial tumor cells [19,20] and we have documented that caspase-3 activity is enhanced in MSC-treated cells at 24 hours [22]."},{"section":"conclusion_and_future_work","t_start":30,"t_end":32,"d_start":76,"d_end":80,"text":"The present studies show that MSC blocks multiple pathways in mouse mammary tumor cells in vitro."},{"section":"figure","t_start":59,"t_end":61,"d_start":34,"d_end":38,"text":"Each bar represents levels in TM6 tumor cells treated with MSC in three different wells. *P < 0.05 compared with 0 hours."},{"section":"figure","t_start":43,"t_end":45,"d_start":231,"d_end":235,"text":"Possible sites of Se-methylselenocysteine (MSC) interaction with components of the Rasâphosphatidylinositol 3-kinaseâAkt (RasâPI3-KâAkt) and RafâMAP kinase/ERK kinaseâERK (RafâMEKâERK) pathways in TM6 mouse mammary tumor cells."}]},"provenance_type":{"database":{"version":"2019-08-16","id":"EuropePMC"}},"is_associated":true,"date_asserted":"2005-01-01T00:00:00Z","evidence_codes":["http://www.targetvalidation.org/evidence/literature_mining","http://purl.obolibrary.org/obo/ECO_0000213"]},"validated_against_schema_version":"1.6.1","access_level":"public","unique_association_fields":{"target_id":"http://identifiers.org/uniprot/O60682","publication_id":"http://europepmc.org/abstract/MED/16168115","disease_id":"http://www.ebi.ac.uk/efo/EFO_0000616"},"literature":{"references":[{"lit_id":"http://europepmc.org/abstract/MED/16168115"}]}}'
json_line = json.loads(json_string[json_string.find('{'):])
# re.sub(r'.*{"type"', '', json_string)

In [ ]:
pmc_id = json_line['evidence']['literature_ref']['lit_id']
print(pmc_id)

In [ ]:
each_sentence = json_line['evidence']['literature_ref']['mined_sentences'][6]
each_sentence

In [ ]:
section = each_sentence['section']
t_start = each_sentence['t_start']
t_end = each_sentence['t_end']
d_start = each_sentence['d_start']
d_end = each_sentence['d_end']
sentence = each_sentence['text']

target = sentence[t_start:t_end+1].strip()
disease = sentence[d_start-1:d_end+1].strip()

print(section)
print(sentence)
print(target)
print(disease)

In [ ]:
import re
def extract_ner_tokens(text):      
  matches=re.findall(r'\"(.+?)\"',text)
  # matches is now ['String 1', 'String 2', 'String3']
  return ', '.join(list(set(matches)))

extract_ner_tokens('Regex should return "String 1" or "String 1" or "String 1" ')


In [ ]:
sentence_list = []
for each_word in sentence.split():
    try:
        sentence_list.append(
            each_word.encode('latin_1').decode('utf-8').strip())  # Each char is a Unicode codepoint.
    except:
        sentence_list.append(re.sub(r'[^\x00-\x7f]', r'', each_word).strip())

sentence = ' '.join(sentence_list)

gene_sentence = Sentence(sentence)
gene_model.predict(gene_sentence)

disease_sentence = Sentence(sentence)
disease_model.predict(disease_sentence)

ml_target = extract_ner_tokens(str(gene_sentence.get_spans('ner')).replace('.','').replace(',',''))
ml_disease = extract_ner_tokens(str(disease_sentence.get_spans('ner')).replace('.','').replace(',',''))

print(ml_target)
print(ml_disease)

In [ ]:
association_sentence = Sentence(sentence)
association_classifier.predict(association_sentence)

if str(association_sentence.labels[0]).split()[0] == 'YGD':
    association_score_ = float(str(association_sentence.labels[0]).split()[1].replace('(', '').replace(')',''))
else:
    association_score_ = 1.0-float(str(association_sentence.labels[0]).split()[1].replace('(', '').replace(')',''))

association_score = format(association_score_, '.2f')
print(association_score)

In [ ]:
sentiment_sentence = Sentence(sentence)
sentiment_classifier.predict(sentiment_sentence)

if str(sentiment_sentence.labels[0]).split()[0] == 'PA':
    sentiment_score_ = float(str(sentiment_sentence.labels[0]).split()[1].replace('(', '').replace(')',''))
else:
    sentiment_score_ = -1.0*float(str(sentiment_sentence.labels[0]).split()[1].replace('(', '').replace(')',''))

sentiment_score = format(sentiment_score_, '.2f')
print(sentiment_score)

In [ ]:
str(sentiment_sentence.labels[0]).split()[1]

In [ ]:
def process_each_split_to_extract_sentences(complete_file_path):
    with open(complete_file_path) as f:
        print(complete_file_path)

        for line in f:
            json_line = json.loads(line[line.find('{'):])
            pmc_id = json_line['evidence']['literature_ref']['lit_id']
#             print(pmc_id)
            try:
                for each_sentence in json_line['evidence']['literature_ref']['mined_sentences']:
                    
                    section = each_sentence['section']
                    t_start = each_sentence['t_start']
                    t_end = each_sentence['t_end']
                    d_start = each_sentence['d_start']
                    d_end = each_sentence['d_end']
                    sentence_ = each_sentence['text']

                    target = sentence_[t_start:t_end+1]
                    disease = sentence_[d_start:d_end+1]
                    
                    ################# Format Sentence ###########################
                    
                    sentence_list = []
                    for each_word in sentence_.split():
                        try:
                            sentence_list.append(
                                each_word.encode('latin_1').decode('utf-8').strip())  # Each char is a Unicode codepoint.
                        except:
                            sentence_list.append(re.sub(r'[^\x00-\x7f]', r'', each_word).strip())
        
                    sentence = ' '.join(sentence_list)
                    sentence = sentence.replace(',' ,' ')
                    
                    ########################### NER #############################
                    
                    gene_sentence = Sentence(sentence)
                    gene_model.predict(gene_sentence)
                    
                    disease_sentence = Sentence(sentence)
                    disease_model.predict(disease_sentence)
                    
                    ml_target = extract_ner_tokens(str(gene_sentence.get_spans('ner')).replace('.','').replace(',',''))
                    ml_disease = extract_ner_tokens(str(disease_sentence.get_spans('ner')).replace('.','').replace(',',''))
            
#                     print(ml_target+ '  ---  ' + ml_disease)
                
                    ########################### Association #############################    
                    
                    
                    association_sentence = Sentence(sentence)
                    association_classifier.predict(association_sentence)

                    if str(association_sentence.labels[0]).split()[0] == 'YGD':
                        association_score_ = float(str(association_sentence.labels[0]).split()[1].replace('(', '').replace(')',''))
                    else:
                        association_score_ = 1.0-float(str(association_sentence.labels[0]).split()[1].replace('(', '').replace(')',''))
    
                    association_score = format(association_score_, '.2f')
#                     print(association_score)
                 
                    ########################### Sentiment #############################    
                    
                    
                    sentiment_sentence = Sentence(sentence)
                    sentiment_classifier.predict(sentiment_sentence)

                    if str(sentiment_sentence.labels[0]).split()[0] == 'PA':
                        sentiment_score_ = float(str(sentiment_sentence.labels[0]).split()[1].replace('(', '').replace(')',''))
                    else:
                        sentiment_score_ = -1.0*float(str(sentiment_sentence.labels[0]).split()[1].replace('(', '').replace(')',''))
    
                    sentiment_score = format(sentiment_score_, '.2f')
#                     print(sentiment_score)
        
                
#                     with open(result_folder + 'Preliminary_ML_OTAR_review.csv', 'a') as sent_file:
#                         sent_file.writelines(sentence+'\n')

                    OTAR_info = [pmc_id, section, target, disease, sentence, ml_target, ml_disease, association_score, sentiment_score]
                    OTAR_data = pd.DataFrame(columns=colNames)

                    OTAR_dict = dict(zip(colNames, OTAR_info))
                    OTAR_CSV = OTAR_data.append(OTAR_dict, ignore_index=True)

                    OTAR_CSV.to_csv(result_folder + 'Preliminary_ML_OTAR_review.csv', sep='\t', encoding='utf-8', index=False, mode='a', header=False)

                    del OTAR_data  # Very important to delete or it will consume the memory

            except:
                print('exception in -- '+pmc_id)
                continue

In [ ]:
process_each_split_to_extract_sentences(file_path)

In [ ]:
# import pandas as pd
names_ = ['PMC_ID','SECTION','EPMC_TARGET','EPMC_DISEASE','SENTENCE','ML_TARGET','ML_DISEASE','ASSOCIATION','SENTIMENT']

df3 = pd.read_csv(result_folder + 'Preliminary_ML_OTAR_review.csv', sep='\t', encoding='utf-8', names=names_)

distributed_set = df3.groupby('SECTION').head(56)

distributed_set.to_csv(result_folder + 'Preliminary_ML_OTAR_review_balanced.csv', sep='\t', encoding='utf-8', index=False, header=True)


In [5]:
from nltk.tokenize import wordpunct_tokenize

text = 'Increased GABAergic tone induces motor incoordination, and extracellular GABA in cerebellum correlates with motor incoordination in rats'

sentence= ' '.join(wordpunct_tokenize(text))

gene_sentence = Sentence(sentence)
gene_model.predict(gene_sentence)

disease_sentence = Sentence(sentence)
disease_model.predict(disease_sentence)

ml_target = gene_sentence.get_spans('ner')
ml_disease = disease_sentence.get_spans('ner')

print(ml_target)
print(ml_disease)

[<GENE-span (3,4,5): "TNF - 1">]
[]


In [ ]:
sentence= Sentence(' '.join(wordpunct_tokenize(text)))
sentence